# Récursivité

L'idée générale d'une fonction récursive est d'identifier un ou plusieurs "cas de base" - des valeurs des données d'entrée pour lesquelles le problème est trivial - et de relier les cas plus complexes au cas de base. En se ramenant par étapes au cas de base, on peut réaliser un processus répétitif et remplacer une boucle.

La formulation d'une solution récursive est souvent plus simple qu'une formulation itérative, mais plus difficile à trouver: c'est une façon particulière de penser, qu'on peut améliorer avec la pratique.

## Technique: concevoir une fonction récursive

Voici une technique générale pour concevoir une fonction récursive:

1. Identifier un paramètre qui définit la _complexité_ du problème, et les cas de base.

2. Écrire la fonction pour différentes valeurs du paramètre / différents niveaux de complexité: f1, f2, f3

3. Chercher à re-écrire la fonction f2 en utilisant f1 , la fonction f3 en utilisant f2... 

4. Généraliser la relation récursive: écrire f(n) en utilisant f(n-1), et incorporer le cas de base, qui sera f(0).

### Exemple détaillé

On veut écrire la fonction _puissance_: ```puissance(x, n)``` calcule $x^n$. 

1. Paramètre de complexité: ce paramètre est en général un nombre entier, et des petites valeurs (0 ou 1) correspondent à un cas où le problème est facile à résoudre (les cas de base). Ce paramètre peut être la valeur d'un paramètre entier, le nombre de chiffres d'un nombre, la longueur d'une liste, etc.

Pour notre exemple, le paramètre ```n``` de la fonction est le paramètre de complexité: pour ```n==0``` la résultat est trivial: $x^0$ vaut 1 pour tout $x$.

2. Fonctions f0, f1, f2: on veut écrire des fonctions où les entrées correspondent aux valeurs 0, 1, 2... du paramètre de complexité: un paramètre de valeur 0, 1, 2, des listes de longueur 0, 1, 2, des nombres à 1, 2, 3 chiffres, etc.

Pour notre exemple, on fixe simplement la valeur de ```n``` à 0, 1, 2:

In [1]:
def puissance0(x):
    return 1

def puissance1(x):
    return x

def puissance2(x):
    return x*x

3. Chercher à re-écrire la fonction f2 en utilisant f1 , la fonction f3 en utilisant f2: ici il faut chercher à ré-écrire chaque fonction en utilisant la précédente, et il faut essayer de toujours l'utiliser de la même façon.

Pour notre exemple on peut écrire:

In [2]:
def puissance0(x):
    return 1

def puissance1(x):
    return puissance0(x)*x

def puissance2(x):
    return puissance1(x)**x

4. Généraliser la relation récursive: écrire f(n) en utilisant f(n-1), et incorporer le cas de base, qui sera f(0).

Pour l'exemple, il est clair que le motif qu'on a utilisé pour puissance1 et puissance2 marche pour tout ```n```. On peut donc écrire:

In [3]:
def puissance(x, n):
    if (n==0):
        return 1
    return puissance(x, n-1)*x

#### Exercice 1

En utilisant la technique ci-dessus, écrire une fonction récursive ```sommeJusqua(n)``` qui calcule la somme des entiers de 1 à n.

## Récursivité terminale

### Problématique
Dans l'implémentation "standard" d'une fonction au niveau de la mémoire (on crée un nouveau bloc sur la pile pour chaque appel de fonction), l'utilisation de processus récursifs est problématique, parce qu'un nouveau bloc (_stack frame_) est créé en mémoire pour chaque appel récursif. 

Par conséquent, l'équivalent récursif d'une boucle répétée 1000 fois sera un processus avec 1000 appels récursifs, et 1000 blocs créés sur la pile. Ceci peut poser problème: c'est une utilisation inefficace de la mémoire et on peut en arriver à épuiser toute la mémoire disponible (débordement de la pile, ou _stack overflow_). 

Il existe un solution à ce problème, qui combine deux choses: une technique de programmation dite récursivité terminale (_tail recursion_), et une optimisation au niveau du compilateur (ou de l'interpréteur) appelée _tail call optimisation_ (TCO).

### Récursivité terminale: exemple et contre-exemple
La récursivité terminale consiste à écrire une fonction récursive de telle manière que le cas récursif **se termine** par l'appel récursif. 

Considérons d'abord la fonction ```puissance``` implémentée ci-dessus: elle n'est **pas** récursive terminale, parce que l'appel récursif ```puissance(x, n-1)``` **ne termine pas** le calcul implémenté par la fonction. En effet, la dernière ligne est:
```
return puissance(x, n-1)*x
````
Le fait que l'appel récursif soit sur cette dernière ligne prête à confusion, mais si on regarde en détail, cette ligne est exécutée en quatre étapes:
1. soustraction ```n-1```
2. appel récursif de ```puissance(x, n-1)```
3. multiplication du résultat par ```x```
4. retour du résultat et fin de la fonction

L'étape (3) est un calcul qui doit se faire entre le l'appel récursif et le retour du r'sultat. Pour que cette fonction soit récursive terminale, il faudrait supprimer cette étape (3). 

Pour un exemple de fonction qui est bien récursive terminale, on peut regarder le cas de la fonction récursive classique qui calcule le PGCD (plus grand dénominateur commun) de deux entiers:

In [4]:
def pgcd(a, b):
    if(a==b):
        return a
    elif(a>b):
        return pgcd(a-b, b)
    else:
        return pgcd(a, b-a)

Ici la fonction ```pgcd``` est récursive terminale. Il y a deux cas récursifs, prenons le premier (a>b): la dernière ligne est alors ```return pgcd(a-b, b)``` et s'exécute comme suit:
1. soustraction ```c=a-b``` 
2. appel récursif de ```pgcd(c, b)```
3. retour de la valeur calculée par l'appel récursif

L'autre cas récursif est similaire: il n'y a aucun calcul entre l'appel récursif et le retour du résultat.

La section suivante explique comment cette propriété peut être exploitée pour optimiser l'usage de la mémoire, et on verra ensuite une technique générale pour convertir une fonction récursive quelconque en fonction récursive terminale.

### Tail-call optimization

Lorsqu'on a une fonction récursive terminale, la version compilée du code peut être optimisée pour que les appels récursifs ne créent pas des nouveaux blocs (_stack frame_) sur la pile. 

Pour comprendre comment ça fonctionne, il faut savoir que le bloc mémoire correspondant à un appel de fonction contient les paramètres de la fonction, les variables locales, et l'adresse de retour. Dans une fonction récursive terminale, au moment de faire l'appel récursif, on a déjà effectué tous les calculs qui utilisaient les paramètres de la fonction, et à part l'adresse de retour, ces données ne sont donc plus nécessaires. On peut donc garder seulement l'adresse de retour (on n'a pas besoin de créer une autre adresse de retour pour l'appel récursif) et on peut éliminer les autres données. On peut donc réutiliser le même espace mémoire pour le stack frame de l'appel récursif. Ainsi, un processus récursif avec une fonction récursive terminale nécessite un espace mémoire fixe, quel que soit le nombre d'appels récursifs.

Ceci dit, pour que le code soit compilé de cette manière, il faut que le compilateur identifie la récursivité terminale, et implémente l'optimisation: cette optimisation est surtout présente dans les langages fonctionnels comme Haskell, Scala, Scheme, F#, etc.

Notons que malgré que les exemples présentés ici utilisent Python, l'implémentation ordinaire du langage (CPython) ne réalise pas cette optimisation. Wikipedia donne une [liste de langages](https://en.wikipedia.org/wiki/Tail_call#By_language) qui implémentent TCO.

### Technique: récursivité terminale

Pour concevoir une fonction récursive terminale, il y a une technique générale qui s'applique presque toujours, tant que la fonction à modifier utilise un seul appel récursif.

L'implémentation récursive d'une fonction a en général la forme suivante:

```
def fonction(parametres):
    if (cas_de_base):
        return valeur_base
    else
        calculs1
        resultat_recursif = fonction(parametres_recursifs)
        resultat_final = calculs2(parametres, resultat_recursif)
        return resultat_final
```
Pour concevoir une version récursive terminale, on introduit en général une deuxième fonction annexe, qui aura un paramètre de plus et sera récursive terminale. La fonction originale sera alors simplement un appel de la fonction annexe:

```
def fonction(parametres):
    return fonction_annexe(parametres, valeur_initiale_a)
    
def fonction_annexe(parametres, a): # parametre supplémentaire a
    if (cas_de_base):
        return a
    else
        nouveau_a = calculs(parametres, a)
        return fonction_annexe(parametres_recursifs, nouveau_a)
```

La clé de cette technique est de déplacer le calcul fait **après l'appel récursif** (représenté par ```calcul2``` dans la fonction originale), et de le faire **au fur et à mesure** du processus récursif: ça peut se faire souvent à l'aide d'un **accumulateur**. 

Pour mieux comprendre, voyons l'exemple de la fonction ```puissance``` vue ci-dessus. Si on détaille le calcul de ```puissance(5, 3)```, on voit que les étapes du calcul seront les suivantes:

1. ``` puissance(5,3)``` va donner ```[...] return puissance(5, 2) * 5```, et on remplace donc:
2. ``` puissance(5, 2) * 5```
2. ``` (puissance(5, 1) * 5) * 5```
2. ``` ((puissance(5, 0) * 5) * 5) * 5```
2. ``` ((1 * 5) * 5) * 5 ```    (cas de base)
3. ``` (5 * 5) * 5 ```
3. ``` 25 * 5 ```
3. ``` 125 ```   (résultat final)

À l'étape 5, on a le retour du dernier appel récursif (qui nous donne 1), qu'on attendait pour pouvoir faire la séquence de multiplications dans l'ordre montré par les parenthèses. Les parenthèses ne sont pas explicites dans le code, mais chaque parenthèse correspond à la fin d'un des appels récursifs. Les différents stack frames pour ces appels récursifs servent essentiellement à garder en mémoire les 4 facteurs du calcul de $5^3$. On comprend donc que pour $x^n$, on va garder en mémoire _n_ + 1 facteurs, et c'est là l'explication de la mémoire nécessaire. 

Cependant, la multiplication étant associative à droite et à gauche, on pourrait aussi faire le produit au fur et à mesure, comme on le ferait dans une boucle:
```
produit =1
for i in range(n):
    produit = produit * 5
```

En déroulant le calcul, on a toujours seulement deux facteurs à la fois en mémoire: 5, et le produit accumulé jusque là:

2. ``` produit = 1```
2. ``` produit = 1 * 5```
2. ``` produit = 5 * 5```
2. ``` produit = 25 * 5```

Les implémentations itératives d'une somme, d'un produit, d'un comptage, etc. utilisent une technique similaire, avec une variable initialisée avant la boucle, et qui "accumule" les résultats des calculs intermédiaires. On appelle donc cette variable un accumulateur.

Dans la version récursive terminale de la fonction ```puissance```, le paramètre supplémentaire passé à la fonction annexe joue le rôle de l'accumulateur (et il est donc initialisé à 1, comme la variable ```produit``` dans la boucle ci-dessus), et on peut rendre explicite la similarité avec la méthode itérative:

In [5]:
def puissance(x, n):
    return puissanceRT(x, n, 1)

def puissanceRT(x, n, produit): #on utilise une fonction annexe récursive terminale
    if (n==0):
        return produit
    produit = produit * x #calcul fait avant l'appel récursif, et résultat passé en paramètre
    return puissanceRT(x, n-1, produit)

#### Exercice 2

Écrire une version récursive terminale de la fonction ```sommeJusqua(n)``` de l'exercice 1.

### Fonctions avec plusieurs appels récursifs

Pour les fonctions qui nécessitent plusieurs appels récursifs, la conception d'une version récursive terminale est nettement plus complexe. 

Ici on parle de fonctions qui font plusieurs appels récursifs __en séquence__, comme la fonction classique qui calcule les termes de la suite de Fibonacci. En revanche, la fonction ```pgcd``` ci-dessus, où il y a deux cas récursifs séparés, ne pose aucun problème: un seul des deux appels sera effectué dans une exécution donnée de la fonction.

Cependant, les fonctions qui utilisent plusieurs appels récursifs  sont souvent problématiques, car pour une "profondeur" de _n_ on peut avoir $2^n$ appels récursifs. Autrement dit, la version récursive "naïve" d'une telle fonction n'est probablement pas efficace et la convertir en récursivité terminale sans complètement modifier l'algorithme n'améliorera pas les choses.

La situation typique où on a des fonctions récursives avec plusieurs appels récursifs est celle des structures de données comme les arbres: il est alors assez difficile de "linéariser" les algorithmes récursifs, et on n'abordera pas cette problématique ici.

## Récursivité sur les listes 
Pour manipuler des listes, on utilise en général des boucles. En programmation fonctionnelle, il nous faut remplacer ces boucles par d'autres techniques. Une qui est souvent appropriée est la récursivité: pour tout algorithme itératif (utilisant une boucle), on peut concevoir un algorithme récursif qui réalise la même fonctionnalité.

### Définition récursive d'une liste
Les deux principales implémentations de listes sont les tableaux (Array) et les listes chaînées (Linked List): ces deux implémentations reflètent deux façon de comprendre les listes. 

Un tableau est représenté en mémoire par des cases mémoires contigües, qui contiennent les éléments de la liste. Pour obtenir le ```n```-ième élément, on a juste besoin de décaler l'adresse du tableau de ```n``` fois la taille d'un élément. 

Une liste chainée est une séquence de "noeuds", chaque noeud étant une paire de la forme : _(valeur, pointeur vers le prochain noeud)_. Ainsi, un pointeur qui pointe vers un noeud pointe aussi vers toute une liste: c'est une vue très récursive d'une liste: _une liste est un élément suivie d'une liste_.

En anglais, le premier élément est souvent appelé _head_, et le "reste" _tail_.
Cette vue récursive d'une liste est en général très utile pour écrire des fonctions récursives sur les listes: le cas récursif sera très souvent un calcul utilisant le _head_ et le _tail_ de la liste. Par exemple, pour un tri par sélection, on sélectionne la plus petite valeur de la liste, on la permute avec _head_, et on effectue ensuite le tri récursif de _tail_.  

En Python, le premier élément d'une liste ```a``` est ```a[0]```, et le "reste" s'écrit ```a[1:]```.

In [6]:
a = [1, 2, 3, 4]
a[0]

1

In [7]:
a[1:]

[2, 3, 4]

### Fonctions récursives sur les listes

Les fonctions récursives sur les listes sont souvent un peu plus complexes que les fonctions récursives sur les nombres, (le fait de manipuler les éléments de la liste ajoute une certain complexité), mais en revanche les fonctions se ressemblent plus. Si on se réfère à la technique générale pour concevoir des fonctions récursives (ci-dessus), on retrouvera ainsi des points communs:
* le _paramètre de complexité_ est en général la longueur de la liste, et les cas de bases sont alors la liste vide ou la liste d'un seul élément. 
* lorsqu'on cherche à se rapprocher du cas de base, on se ramène toujours vers une liste plus courte, qui sera très souvent la liste _tail_. Le calcul dans le cas récursif utilise donc presque toujours la décomposition _head_+_tail_. 

#### Exemple: calculer la somme les éléments d'une liste

On veut écrire une fonction récursive ```somme(liste)``` qui retourne la somme des éléments d'une liste de nombres.

Comme on s'y attendrait, la longueur de la liste est le paramètre de complexité du problème. On peut considérer que la fonction n'est pas définie pour une liste vide (on pourrait aussi dire que la somme vaut 0). Le cas de base est donc une liste d'un élément: la somme de ses éléments est la valeur de l'élément. 

Écrivons d'abord les fonctions somme1, somme2, somme3 qui fonctionneront pour des listes de longueur 1, 2, 3:

In [8]:
def somme1(liste): #par hypothèse, liste contient exactement 1 élément
    return liste[0]

def somme2(liste): #par hypothèse, liste contient exactement 2 éléments
    return liste[0] + liste[1]

def somme3(liste): #par hypothèse, liste contient exactement 2 éléments
    return liste[0] + liste[1] + liste[2]

À présent, on doit ré-écrire f2 en utilisant f1, et f3 en utilisant f2. 
Comme indiqué au-dessus, on va essayer d'utiliser la décomposition _head_ - _tail_:

In [9]:
def somme1(liste): #par hypothèse, liste contient exactement 1 élément
    return liste[0] 

def somme2(liste): #par hypothèse, liste contient exactement 2 éléments
    return liste[0] + somme1(liste[1:]) # liste[1] est l'unique élément de tail (liste[1:]), donc aussi la somme de tail

def somme3(liste): #par hypothèse, liste contient exactement 2 éléments
    return liste[0] + somme2(liste[1:])  #liste[1]+liste[2] est aussi la somme des éléments de tail (une liste à deux éléments)

On peut constater que le code des trois fonctions est vraiment similaire, et la généralisation est assez simple.
En séparant le cas de base, on obtient une fonction qui marche pour des listes de longueur quelconque:

In [10]:
def somme(liste):
    if(len(liste)==0):
        return 0
    else:
        return liste[0] + somme(liste[1:]) 

On teste:

In [11]:
b = [2, 4, 1, 5]

In [12]:
somme(b)

12

### Exercices

#### Exercice 3
Écrire une fonction récursive ```contient(liste, e)``` qui retourne ```True``` si la liste contient l'élément ```e```, sinon ```False```.

#### Exercice 4

Écrire une fonction récursive ```elementsPairs(liste)``` qui prend en entrée une liste de nombres entiers positifs et retourne le nombre d'éléments de la liste qui sont pairs. Exemple: ```elementsPairs([7, 14, 2, 0, 39])``` retourne 3.

### Fonctions récursives terminales sur les listes

Le principe de la récursivité terminale peut aussi s'appliquer aux fonctions sur les listes.

On utilisera ici aussi une fonction annexe avec un mécanisme d'accumulateur, où on va chercher à prendre les calculs qui sont faits ```après``` l'appel récursif, et les faire ```avant``` l'appel récursif. La nature de l'accumulateur dépend du but de la fonction: si la fonction doit retourner une liste (exemple: ```reverse```, qui donne la liste d'entrée à l'envers), alors l'accumulateur est une liste. Si le résultat attendu est un nombre (exemple: compter le nombre d'éléments), alors l'accumulateur sera un nombre.

**Exemple** Une version récursive terminale de la fonction ```somme``` sera la suivante:

In [42]:
def sommeRT(liste):
    return somme_helper(liste, 0)  # valeur initiale de la somme = 0

def somme_helper(liste, acc):
    if (liste==[]):
        return acc
    else:
        return somme_helper(liste[1:], liste[0] + acc)

Testons:

In [43]:
sommeRT([4, 6, 1, 1])

12

#### Exercice 5

**5.1**
Écrire une fonction récursive ```maxR``` qui trouve la valeur maximale d'une liste de nombres.

In [38]:
def maxR(liste):
    ...

**5.2**
Écrire une version récursive terminale ```maxRT``` qui trouve la valeur maximale en utilisant un mécanisme récursif terminal.

In [36]:
def maxRT(liste):
    ...
    
def maxRT_helper(liste, acc):
    ...
